In [ ]:
# hybrid_qor_train.py
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import os
import itertools
import plotly.graph_objs as go

# 1. Load and Clean Data
csv_path = r"D:\Data\NYC\KINZ\KINECT_ACC_dataset_with_qor15_2025-05-27_14-29PM.csv"
df = pd.read_csv(csv_path, low_memory=False)
joint_cols = [col for col in df.columns if any(j in col for j in ['_X', '_Y', '_Z'])]
df = df.dropna(subset=joint_cols + ['QoR_class', 'footfall_event_times', 'accel_energy_total'])

# 2. Patient-wise Split
patient_map = df.groupby('patientID')['QoR_class'].first()
class_0 = patient_map[patient_map == 0.0].index.tolist()
class_1 = patient_map[patient_map == 1.0].index.tolist()

def split(patients):
    train, temp = train_test_split(patients, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)
    return train, val, test

train_0, val_0, test_0 = split(class_0)
train_1, val_1, test_1 = split(class_1)
train_ids = train_0 + train_1
val_ids = val_0 + val_1
test_ids = test_0 + test_1

train_df = df[df['patientID'].isin(train_ids)]
val_df = df[df['patientID'].isin(val_ids)]
test_df = df[df['patientID'].isin(test_ids)]

# === Fixed Joint Order ===
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# === Matching Edge List ===
edges = torch.tensor([
    [0, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [7, 8], [8, 9],
    [9, 10], [10, 11], [10, 12], [11, 13], [12, 14], [13, 15], [14, 16],
    [15, 17], [16, 18], [17, 19], [18, 20], [19, 21], [20, 22], [17, 23],
    [18, 24], [10, 25], [25, 26], [26, 27], [26, 28], [26, 29], [26, 30], [26, 31]
]).t().contiguous()

class KinectTemporalGraphDataset(torch.utils.data.Dataset):
    def __init__(self, df, seq_len=16, stride=1):
        self.graphs = []
        self.temporal_feats = []
        self.labels = []

        for _, patient in tqdm(df.groupby('patientID'), desc="Creating graph dataset"):
            patient = patient.sort_values('t_uniform').reset_index(drop=True)
            for start in range(0, len(patient) - seq_len + 1, stride):
                seq_graphs = []
                seq_feats = []
                for i in range(seq_len):
                    row = patient.iloc[start + i]
                    coords = torch.tensor(
                        [[row[f"{j}_X"], row[f"{j}_Y"], row[f"{j}_Z"]] for j in joints],
                        dtype=torch.float
                    )

                    # === Normalize: center to PELVIS ===
                    root = coords[joints.index('PELVIS')].clone()
                    coords = coords - root

                    # === Normalize: scale to average distance from root ===
                    scale = coords.norm(dim=1).mean().clamp(min=1e-5)
                    coords = coords / scale

                    x = coords
                    seq_graphs.append(Data(x=x, edge_index=edges.clone()))
                    seq_feats.append([row['footfall_event_times'], row['accel_energy_total']])
                label = torch.tensor([patient.iloc[start + seq_len // 2]['QoR_class']], dtype=torch.float)

                self.graphs.append(seq_graphs)
                self.temporal_feats.append(torch.tensor(seq_feats, dtype=torch.float))
                self.labels.append(label)

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, i):
        return self.graphs[i], self.temporal_feats[i], self.labels[i]

def temporal_collate(batch):
    graphs, feats, labels = zip(*batch)
    return list(graphs), torch.stack(feats), torch.tensor(labels, dtype=torch.float)

class HybridGCNBiLSTM(nn.Module):
    def __init__(self, gcn_hidden=64, lstm_hidden=128, dropout=0.3):
        super().__init__()
        self.gcn1 = GCNConv(3, gcn_hidden)
        self.gcn2 = GCNConv(gcn_hidden, gcn_hidden)
        self.graph_lstm = nn.LSTM(gcn_hidden, lstm_hidden, batch_first=True, bidirectional=True)
        self.feat_lstm = nn.LSTM(2, lstm_hidden, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(lstm_hidden * 4, 1)
        self.dropout = dropout

    def forward(self, graph_seq, feat_seq):
        device = next(self.parameters()).device
        batch_embeddings = []
        for g_seq in graph_seq:
            seq_embed = []
            for g in g_seq:
                g = g.to(device)
                x = F.relu(self.gcn1(g.x, g.edge_index))
                x = F.dropout(x, self.dropout, self.training)
                x = F.relu(self.gcn2(x, g.edge_index))
                pooled = global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long).to(device)).squeeze(0)
                seq_embed.append(pooled)
            batch_embeddings.append(torch.stack(seq_embed))

        graph_input = torch.stack(batch_embeddings).to(device)  # [B, T, gcn_hidden]
        feat_input = feat_seq.to(device)  # [B, T, 2]

        graph_input = graph_input.contiguous()
        feat_input = feat_input.contiguous()

        _, (h_g, _) = self.graph_lstm(graph_input)
        _, (h_f, _) = self.feat_lstm(feat_input)

        concat = torch.cat([h_g[-2], h_g[-1], h_f[-2], h_f[-1]], dim=-1)  # BiLSTM last hidden states
        return self.fc(concat).squeeze()


def train_epoch(model, loader, opt, criterion):
    model.train()
    total_loss = 0
    for graphs, feats, labels in tqdm(loader, desc="Train", leave=False):
        opt.zero_grad()
        preds = model(graphs, feats)
        loss = criterion(preds, labels.to(preds.device))
        loss.backward()
        opt.step()
        total_loss += loss.item() * len(labels)
    return total_loss / len(loader.dataset)


def eval_epoch(model, loader, criterion):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for graphs, feats, labels in loader:
            preds = model(graphs, feats)
            preds_all.extend(torch.sigmoid(preds).view(-1).cpu().numpy())  # <- fixed line
            labels_all.extend(labels.view(-1).numpy())
    bin_preds = [1 if p > 0.5 else 0 for p in preds_all]
    acc = accuracy_score(labels_all, bin_preds)
    prec = precision_score(labels_all, bin_preds, zero_division=0)
    rec = recall_score(labels_all, bin_preds)
    f1 = f1_score(labels_all, bin_preds)
    return acc, prec, rec, f1



def run_tuning():
    param_grid = {
        'seq_len': [2, 4, 32, 64],
        'batch_size': [4,8],
        'gcn_hidden': [64,128],
        'lstm_hidden': [128,256],
        'dropout': [0.3, 0.5]
    }
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with open("tuning_log.txt", "w") as log:
        log.write("Hyperparameter Tuning\n" + "="*50 + "\n")

    for params in itertools.product(*param_grid.values()):
        p = dict(zip(param_grid.keys(), params))
        train_ds = KinectTemporalGraphDataset(train_df, seq_len=p['seq_len'])
        val_ds = KinectTemporalGraphDataset(val_df, seq_len=p['seq_len'])
        test_ds = KinectTemporalGraphDataset(test_df, seq_len=p['seq_len'])
        train_dl = DataLoader(train_ds, batch_size=p['batch_size'], shuffle=True, collate_fn=temporal_collate)
        val_dl = DataLoader(val_ds, batch_size=p['batch_size'], shuffle=False, collate_fn=temporal_collate)
        test_dl = DataLoader(test_ds, batch_size=p['batch_size'], shuffle=False, collate_fn=temporal_collate)

        model = HybridGCNBiLSTM(**{k: p[k] for k in ['gcn_hidden', 'lstm_hidden', 'dropout']}).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        labels = [l.item() for _, _, l in train_ds]
        weight = compute_class_weight('balanced', classes=np.array([0., 1.]), y=labels)
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weight[1]], dtype=torch.float).to(device))

        best_f1, patience = 0, 0
        for epoch in range(30):
            train_epoch(model, train_dl, opt, criterion)
            acc, prec, rec, f1 = eval_epoch(model, val_dl, criterion)
            if f1 > best_f1:
                best_f1 = f1
                patience = 0
                torch.save(model.state_dict(), "best_model.pt")
            else:
                patience += 1
                if patience > 10: break

        model.load_state_dict(torch.load("best_model.pt"))
        acc, prec, rec, f1 = eval_epoch(model, test_dl, criterion)
        with open("tuning_log.txt", "a") as log:
            log.write(f"Params: {p}\nACC: {acc:.4f} PREC: {prec:.4f} REC: {rec:.4f} F1: {f1:.4f}\n{'-'*40}\n")

if __name__ == '__main__':
    run_tuning()
